In [1]:
# TODO : Include the PYQGIS imports for the plugin
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math
import subprocess
import shutil
import os
import sys

pymeos_initialize()
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "PyMEOS_demo"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "trajectory"

SRID = 4326




In [2]:
Time_granularities = {
                # "MILLISECOND" : timedelta(milliseconds=1),
                    "SECOND" : timedelta(seconds=1),
                    "MINUTE" : timedelta(minutes=1),
                #   "HOUR" : timedelta(hours=1),
                }

In [3]:
class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": DATABASE_NAME,
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = TPOINT_TABLE_NAME
            self.id_column_name = TPOINT_ID_COLUMN_NAME
            self.tpoint_column_name = TPOINT_COLUMN_NAME               
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
            
        except Exception as e:
            pass

    
    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass
        

    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax, time_granularity, start_date):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        
        try:
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))

            if time_granularity == "SECOND":
                time_value = 1
            elif time_granularity == "MINUTE":
                time_value = 60
            
            # return [self.tpoint_column_name, self.id_column_name, ids_str, xmin, ymin, xmax, ymax, pstart, pend, time_granularity, start_date, time_value]
            query = f"""WITH trajectories as (
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        ) as trajectory
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str})),

                    resampled as (

                    SELECT tsample(traj.trajectory, INTERVAL '1 {time_granularity}', TIMESTAMP '{start_date}')  AS resampled_trajectory
                        FROM 
                            trajectories as traj)
				
                    SELECT
                            EXTRACT(EPOCH FROM (startTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS start_index ,
                            EXTRACT(EPOCH FROM (endTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS end_index,
                            rs.resampled_trajectory
                    FROM resampled as rs ;"""
        
            self.cursor.execute(query)
       
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            print(query)
            print(e)


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()

In [4]:
db = Database_connector()
GRANULARITY = Time_granularities["MINUTE"]
start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY )

timestamps = [start_date + i * GRANULARITY for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [5]:
len(timestamps)

1439

# Multiprocessing

In [9]:
import numpy as np
import multiprocessing
from shapely.geometry import Point
def create_matrix(result_queue, begin_frame, end_frame, TIME_DELTA_SIZE, PERCENTAGE_OF_OBJECTS, x_min, y_min, x_max, y_max, timestamps, total_frames, GRANULARITY, db):
    # a  =f"all parameters : {begin_frame, end_frame, TIME_DELTA_SIZE, PERCENTAGE_OF_OBJECTS, x_min, y_min, x_max, y_max, len(timestamps), total_frames, GRANULARITY, db}"
    # result_queue.put(a) 

    print(result_queue.get()) 
    # p_start = timestamps[begin_frame]
    # p_end = timestamps[end_frame]
    # # print(p_start, p_end, x_min, y_min, x_max, y_max)
    # # now_db = time.time()
    


    # rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max, GRANULARITY, start_date)    
    # # print(f"Time to fetch subset of tpoints: {time.time() - now_db} seconds\n")
            
    # empty_point_wkt = Point().wkt  # "POINT EMPTY"
    # matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)
    

    # # now = time.time()

    # for i in range(len(rows)):
    #     if rows[i][2] is not None:
    #         try:
    #             traj_resampled = rows[i][2]

    #             start_index = rows[i][0] - begin_frame
    #             end_index = rows[i][1] - begin_frame
    #             values = np.array([point.wkt for point in traj_resampled.values()])
    #             matrix[i, start_index:end_index+1] = values
        
    #         except:
    #             continue
    matrix = np.full((10, 10), 0, dtype=object)
    result_queue.put(matrix)
    

# Using Manager's queue
# manager = multiprocessing.Manager()
result_queue = multiprocessing.Queue()
result_queue.put("Hello")
# Creating and starting a new process to generate the matrix
process = multiprocessing.Process(target=create_matrix, args=(result_queue, 0, 59, 60, 0.1, -180, -90, 180, 90, timestamps,len(timestamps), 'SECOND',db))
process.start()
# while(True):

    
process.join()  # Wait for the process to complete
a = result_queue.get()
print(a)
print(type(a))
print(len(a))
print(a.shape)
print("Process ended")


Hello
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
<class 'numpy.ndarray'>
10
(10, 10)
Process ended
